In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from googleapiclient.discovery import build
from dotenv import load_dotenv
import os

In [6]:
load_dotenv()
API_KEY = os.getenv('YOUTUBE_API_KEY1')
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [9]:
def search_news_channels():
    # Search for channels related to news in the US (can include any related keywords like 'news', 'journalism')
    request = youtube.search().list(
        part='snippet',
        type='channel',           # Search for channels
        q='news',                  # Search for "news" related channels
        regionCode='US',           # Limit to the US
        maxResults=25             # Limit the number of results to 10
    )
    response = request.execute()
    return response

# Fetch the top 10 news channels
news_channels = search_news_channels()

In [11]:
# Print the channel info (you can modify this to display only the channel IDs and names)
for channel in news_channels['items']:
    channel_id = channel['snippet']['channelId']
    channel_title = channel['snippet']['channelTitle']
    print(f"{channel_title} - {channel_id}")

Fox News - UCXIJgqnII2ZOINSWNOGFThA
ABC News - UCBi2mrWuNuyYy4gbM6fU18Q
News - UCYfdidRxbB8Qhf0Nx7ioOYw
BBC News - UC16niRr50-MSBwiO3YDb3RA
日テレNEWS - UCuTAXTexrhetbOe3zgskJBQ
にゅうちゅうぶ | NEWS official - UCj9f2amb1D0RuTNrBQ9GJoQ
Sky News - UCoMdktPbSTixAyNGwb-UYkQ
ABS-CBN News - UCE2606prvXQc_noEqKxVJXA
TVBS NEWS - UC5nwNW4KdC0SzrhF9BXEYOQ
NEWS AM - UCDv-XtfgNGHXcpwu4ab0WnA
NBC News - UCeY0bbntWzzVIaj2z3QigXg
9 News Australia - UCIYLOcEUX6TbBo7HQVF2PKA
無綫新聞 TVB NEWS Official - UC_ifDTtFAcsj-wJ5JfM27CQ
Geo News - UC_vt34wimdCzdkrzVejwX9g
News - Topic - UCte8ywmzIMWUT3K6rH4H1XQ
CS2 NEWS HUB - UCr4T-eq-wGMZQJiARI0nC_Q
Good News Circle - Topic - UC79umPRGQIo23TpAyfpv48w
Good News - Topic - UCrLbXfsTlSlU4eZYiAXNcsQ
LLA News - China News - UCzgw6gUjklNbJejbuWVGM7Q
NEWS - Topic - UCbYM7oBcZNzqQp98PiF5HKw
Health News - UCn371zWk5jljg-ycIXkEUSA
Bad News Blues Band - Topic - UCIJLo1rxpNoPaoJxll26nsg
News From Nowhere - Topic - UCvOJ4neMIuwsAvoNV82sCPA
ATLANTA FALCONS NEWS TODAY FANS - UCsATXVDHDOla

In [34]:
def get_channel_stats(channel_id):
    request = youtube.channels().list(
        part='snippet,statistics',
        id=channel_id
    )
    response = request.execute()
    stats = response['items'][0]['statistics']
    return {
        'channel_id': channel_id,
        'title': response['items'][0]['snippet']['title'],
        'subscribers': stats.get('subscriberCount', 'hidden'),
        'total_views': stats.get('viewCount'),
        'video_count': stats.get('videoCount')
    }

In [35]:
def get_uploads_playlist_id(channel_id):
    request = youtube.channels().list(
        part='contentDetails',
        id=channel_id
    )
    response = request.execute()
    return response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

def get_video_ids_from_playlist(playlist_id, max_videos=5):
    video_ids = []
    request = youtube.playlistItems().list(
        part='contentDetails',
        playlistId=playlist_id,
        maxResults=max_videos
    )
    response = request.execute()
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
    return video_ids


In [36]:
def get_video_stats(video_id):
    request = youtube.videos().list(
        part='snippet,statistics',
        id=video_id
    )
    response = request.execute()
    data = response['items'][0]
    return {
        'title': data['snippet']['title'],
        'published_at': data['snippet']['publishedAt'],
        'views': data['statistics'].get('viewCount'),
        'likes': data['statistics'].get('likeCount'),
        'comments': data['statistics'].get('commentCount')
    }


In [37]:
def get_video_comments(video_id, max_comments=100):
    comments = []
    request = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        maxResults=100,  # YouTube limits this per page
        textFormat='plainText'
    )
    response = request.execute()
    
    for item in response['items']:
        top_comment = item['snippet']['topLevelComment']['snippet']
        comments.append({
            'author': top_comment['authorDisplayName'],
            'profile_image': top_comment['authorProfileImageUrl'],
            'text': top_comment['textDisplay'],
            'like_count': top_comment['likeCount'],
            'published_at': top_comment['publishedAt']
        })

    return comments[:max_comments]


In [38]:
channel_id = 'UCXIJgqnII2ZOINSWNOGFThA'
channel_stats = get_channel_stats(channel_id)
print(channel_stats)

{'channel_id': 'UCXIJgqnII2ZOINSWNOGFThA', 'title': 'Fox News', 'subscribers': '13500000', 'total_views': '20410513451', 'video_count': '119469'}


In [39]:
playlist_id = get_uploads_playlist_id(channel_id)
print("Uploads Playlist ID:", playlist_id)

Uploads Playlist ID: UUXIJgqnII2ZOINSWNOGFThA


In [40]:
video_ids = get_video_ids_from_playlist(playlist_id, max_videos=3)
print("Video IDs:", video_ids)

Video IDs: ['fyzWSABknTs', '6Em4QyUuLUU', 'HDhVzem6iQw']


In [41]:
video_stats = get_video_stats(video_ids[0])
print(video_stats)

{'title': 'This is who Democrats are protecting: Hannity', 'published_at': '2025-05-06T02:00:39Z', 'views': '535', 'likes': '71', 'comments': '35'}


In [42]:
comments = get_video_comments(video_ids[0], max_comments=5)
for c in comments:
    print(c['author'], "->", c['text'][:100])

@FoxNews -> Read more: https://www.foxnews.com/politics/democrat-hank-johnson-draws-holocaust-comparison-while-b
@Islam__Islam -> If you ever find yourself questioning your existence or you want answers about what your purpose is 
@jasonraynor9538 -> AOC gots - Too- Go ! They are anti everything American .
@onikomehrabi264 -> Show the maps and propaganda that was put out in other countries telling people to come to America.
@dannysart3990 -> State media T.V scaring old people. But not reporting on Trump's crypto scamming.
